In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
import pandas as pd

In [2]:
sc = SparkContext('local')
sqlCtx = SQLContext(sc)

22/07/27 14:22:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [119]:
emp =[('홍길동',1),('이순신',2),
      ('임꺽정',3),('김철수',3),('김철수1',5)]
dept = [('개발',1), ('연구',2),
        ('영업',3),('기획',4) ]

In [126]:
empRdd = sc.parallelize(emp)
empRdd

ParallelCollectionRDD[224] at readRDDFromFile at PythonRDD.scala:274

In [127]:
empDF1 = empRdd.toDF(['name', 'deptid'])
empDF1.show()

+-------+------+
|   name|deptid|
+-------+------+
| 홍길동|     1|
| 이순신|     2|
| 임꺽정|     3|
| 김철수|     3|
|김철수1|     5|
+-------+------+



In [128]:
empDF2 = sqlCtx.createDataFrame(emp,['name', 'deptid'])
empDF2.show()

+-------+------+
|   name|deptid|
+-------+------+
| 홍길동|     1|
| 이순신|     2|
| 임꺽정|     3|
| 김철수|     3|
|김철수1|     5|
+-------+------+



In [7]:
birthDF = sqlCtx.read.csv('data/births.txt')
birthDF.show()

+----+-------+-------+
| _c0|    _c1|    _c2|
+----+-------+-------+
|1880|  90993| 110491|
|1881|  91954| 100745|
|1882| 107850| 113688|
|1883| 112321| 104629|
|1884| 129022| 114445|
|1885| 133055| 107800|
|1886| 144535| 110784|
|1887| 145982| 101414|
|1888| 178627| 120853|
|1889| 178366| 110584|
|1890| 190377| 111025|
|1891| 185482| 101196|
|1892| 212346| 122037|
|1893| 212906| 112317|
|1894| 222922| 115772|
|1895| 233630| 117398|
|1896| 237920| 119570|
|1897| 234202| 112758|
|1898| 258770| 122693|
|1899| 233023| 106212|
+----+-------+-------+
only showing top 20 rows



In [8]:
birthDF.toPandas() # rdd collect　처럼　조심해서　사용

,_c0,_c1,_c2
0,1880,90993,110491
1,1881,91954,100745
2,1882,107850,113688
3,1883,112321,104629
4,1884,129022,114445
...,...,...,...
127,2007,1919408,2072139
128,2008,1887234,2036289
129,2009,1832925,1979303
130,2010,1772738,1913851


In [129]:
empDF = sqlCtx.createDataFrame(emp,['name', 'deptid'])
empDF.show()

+-------+------+
|   name|deptid|
+-------+------+
| 홍길동|     1|
| 이순신|     2|
| 임꺽정|     3|
| 김철수|     3|
|김철수1|     5|
+-------+------+



In [130]:
empDF.createOrReplaceTempView('emptable')

In [11]:
empDF.printSchema() #long은　숫자

root
 |-- name: string (nullable = true)
 |-- deptid: long (nullable = true)



### hive sql 구문　문법을　따른다： standard sql + hive sql

In [33]:
# sql = 'select * from emptable'
# sql = 'select name from emptable'
# sql = 'select name,deptid from emptable'
# sql = 'select * from emptable where deptid>=2'
# sql = 'select * from emptable where name="이순신"'
# sql = 'select * from emptable where name>="이순신"'
# sql = 'select * from emptable where deptid=1 or deptid=5 or deptid==3'
# sql = 'select * from emptable where deptid in(1,3,5)'
# sql = 'select * from emptable where deptid>=2 and deptid<=4'
# sql = 'select * from emptable where deptid between 2 and 4'

# like : 문자열　컬럼검색， %_
# sql = 'select * from emptable where name like "김%"' #김으로시작
# sql = 'select * from emptable where name like "%수"' #수로끝남
# sql = 'select * from emptable where name like "%수%"' #수를포함
# sql = 'select * from emptable where name like "%수_"' #수포함뒤에한글자

# rlike : 문자열　검색　정규식 ： ^ $ [] () ? * + \w \d \W \D \s \S
# sql = 'select * from emptable where name rlike "수"' #수를포함
# sql = 'select * from emptable where name rlike "^김"' #김으로시작
# sql = 'select * from emptable where name rlike "수$"' #수로끝남
# sql = 'select * from emptable where name rlike "[이김]"' #이，김포함
sql = 'select * from emptable where name rlike "순신|꺽정"' #순신이나꺽정포함
empDF1 = sqlCtx.sql(sql)
empDF1.show()

+------+------+
|  name|deptid|
+------+------+
|이순신|     2|
|임꺽정|     3|
+------+------+



In [13]:
df = empDF.toPandas()
df

,name,deptid
0,홍길동,1
1,이순신,2
2,임꺽정,3
3,김철수,3
4,김철수1,5


In [14]:
df[(df['deptid']>=2) & (df['deptid']<=4)]

,name,deptid
1,이순신,2
2,임꺽정,3
3,김철수,3


In [15]:
df[df['deptid'].between(2,4)]

,name,deptid
1,이순신,2
2,임꺽정,3
3,김철수,3


In [16]:
df.query('deptid >= 2 and deptid <= 4')

,name,deptid
1,이순신,2
2,임꺽정,3
3,김철수,3


In [32]:
df[df['name'].str.contains('[이김]')]

,name,deptid
1,이순신,2
3,김철수,3
4,김철수1,5


In [46]:
# 이름에　＇김＇　글자가　있거나　deptid 2 또는 5인　데이터를　검색하시오．
sql = 'select * from emptable where name like "%김%" or deptid rlike "2|5"'
empDF1 = sqlCtx.sql(sql)
empDF1.show()

+-------+------+
|   name|deptid|
+-------+------+
| 이순신|     2|
| 김철수|     3|
|김철수1|     5|
+-------+------+



### 정렬

In [58]:
# sql = 'select * from emptable order by name' #오름차순
# sql = 'select * from emptable order by name desc' #내림차순
# sql = 'select * from emptable order by deptid, name' #deptid>name　오름차순
# sql = 'select * from emptable order by deptid limit 2' #
# sql = 'select * from emptable limit 2' #head랑비슷
sql = 'select * from emptable where deptid>=3 order by deptid desc' #
empDF1 = sqlCtx.sql(sql)
empDF1.show()

+-------+------+
|   name|deptid|
+-------+------+
|김철수1|     5|
| 임꺽정|     3|
| 김철수|     3|
+-------+------+



### 산술 연산 : + - * / ()

In [62]:
# sql = 'select name, deptid, deptid*2 from emptable'
sql = 'select name, deptid, (deptid+2)*3 as dp from emptable'
empDF1 = sqlCtx.sql(sql)
empDF1.show()

+-------+------+---+
|   name|deptid| dp|
+-------+------+---+
| 홍길동|     1|  9|
| 이순신|     2| 12|
| 임꺽정|     3| 15|
| 김철수|     3| 15|
|김철수1|     5| 21|
+-------+------+---+



In [82]:
# name salary 컬럼명을 가지는 데이터프레임을 만드시요.
data1 = ['홍길동,1000','이순신,2000','임꺽정,3000',
         '김철수,4000','이황,5000','이이,6000']

In [86]:
empRdd = sc.parallelize(data1)
empRdd.collect()

['홍길동,1000', '이순신,2000', '임꺽정,3000', '김철수,4000', '이황,5000', '이이,6000']

In [90]:
rdd = empRdd.map(lambda v:v.split(','))

In [93]:
# empDF = empRdd.toDF()
empDF = rdd.toDF(['name','salary'])
empDF.show()

+------+------+
|  name|salary|
+------+------+
|홍길동|  1000|
|이순신|  2000|
|임꺽정|  3000|
|김철수|  4000|
|  이황|  5000|
|  이이|  6000|
+------+------+



In [94]:
df = empDF.toPandas()
df

,name,salary
0,홍길동,1000
1,이순신,2000
2,임꺽정,3000
3,김철수,4000
4,이황,5000
5,이이,6000


###　연습문제
1. 급여가 가장높은 name과 salary를 출력하시요
2. name, salary, tax 를 출력하시요 (tax는 급여에서 세금 3.3을 제한값)
3. name 에 '철'이 포함된  name,salary 를 출력하시요
4. salary top 5 인 name, salary 출력하시요
5. salary  2000  과 4000 사이의 데이터를 출력하시요

In [97]:
empDF.createOrReplaceTempView('emptable')

In [99]:
# 급여가 가장높은 name과 salary를 출력하시요
sql = 'select * from emptable order by salary desc limit 1' #
empDF = sqlCtx.sql(sql)
empDF.show()

+----+------+
|name|salary|
+----+------+
|이이|  6000|
+----+------+



In [101]:
# name, salary, tax 를 출력하시요 (tax는 급여에서 세금 3.3을 제한값)
sql = 'select name, salary, salary-salary*0.033 as tax from emptable' #
empDF = sqlCtx.sql(sql)
empDF.show()

+------+------+------+
|  name|salary|   tax|
+------+------+------+
|홍길동|  1000| 967.0|
|이순신|  2000|1934.0|
|임꺽정|  3000|2901.0|
|김철수|  4000|3868.0|
|  이황|  5000|4835.0|
|  이이|  6000|5802.0|
+------+------+------+



In [111]:
# name 에 '철'이 포함된 name,salary 를 출력하시요
sql = 'select name, salary from emptable where name rlike "[철]"' #
empDF = sqlCtx.sql(sql)
empDF.show()

+------+------+
|  name|salary|
+------+------+
|김철수|  4000|
+------+------+



In [115]:
# salary top 5 인 name, salary 출력하시요
sql = 'select name, salary from emptable order by salary desc limit 5' #
empDF = sqlCtx.sql(sql)
empDF.show()

+------+------+
|  name|salary|
+------+------+
|  이이|  6000|
|  이황|  5000|
|김철수|  4000|
|임꺽정|  3000|
|이순신|  2000|
+------+------+



In [117]:
# salary 2000 과 4000 사이의 데이터를 출력하시요
sql = 'select name, salary from emptable where salary >=2000 and salary <=4000' #
empDF = sqlCtx.sql(sql)
empDF.show()

+------+------+
|  name|salary|
+------+------+
|이순신|  2000|
|임꺽정|  3000|
|김철수|  4000|
+------+------+



### 집계함수（max, min, avg, sum, count)함수（숫자타입)

In [135]:
emp =[('홍길동',1),('이순신',2),
      ('임꺽정',3),('김철수',3),('김철수1',5)]
dept = [('개발',1), ('연구',2),
        ('영업',3),('기획',4) ]

In [136]:
empRdd = sc.parallelize(emp)
empRdd

ParallelCollectionRDD[246] at readRDDFromFile at PythonRDD.scala:274

In [137]:
empDF1 = empRdd.toDF(['name', 'deptid'])
empDF1.show()

+-------+------+
|   name|deptid|
+-------+------+
| 홍길동|     1|
| 이순신|     2|
| 임꺽정|     3|
| 김철수|     3|
|김철수1|     5|
+-------+------+



In [138]:
empDF1.createOrReplaceTempView('emptable')

In [141]:
# sql = "select max(deptid) from emptable"
# sql = "select min(deptid) from emptable"
# sql = "select sum(deptid) from emptable"
# sql = "select avg(deptid) from emptable"
# sql = "select count(deptid) from emptable"
sql = "select sum(deptid),avg(deptid) from emptable"
empDF1 = sqlCtx.sql(sql)
empDF1.show()

+-----------+-----------+
|sum(deptid)|avg(deptid)|
+-----------+-----------+
|         14|        2.8|
+-----------+-----------+



### 조건연산자

In [146]:
sql = '''select name, deptid,
case
when deptid<=2 then 'small' 
when deptid<=4 then 'middle'
else 'big'
end as d
from emptable
'''
empDF1 = sqlCtx.sql(sql)
empDF1.show()

+-------+------+------+
|   name|deptid|     d|
+-------+------+------+
| 홍길동|     1| small|
| 이순신|     2| small|
| 임꺽정|     3|middle|
| 김철수|     3|middle|
|김철수1|     5|   big|
+-------+------+------+

